# Setting-up the computational stage {#sec-20230402140105}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}$ custom macros.

This chapter lays the ground for all symbolic calculations that are to follow. The [SymPy](https://www.sympy.org) library is imported and initialized in @sec-20230103084732. Then, the energy of the system is rewritten in a more convenient form in @sec-20230208081549.

## Importing `Sympy` {#sec-20230103084732}

We will use the [Python](https://www.python.org) library [SymPy](https://www.sympy.org) for symbolic mathematics and rely also on the [IPython](https://ipython.org/) library (in particular, [IPython.display.Math](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.Math)) for pretty $\LaTeX$ output. Some useful functions are defined in the `lsk.py` module, which will be systematically imported.

Without loss of generality, it will be assumed in all symbolic calculations that $\lambda_0 = 0$ and $u_0 = 0$. The general case $\lambda_0 \neq 0$ and $u_0 \neq 0$ is readily recovered through the substitution $\lambda \leftrightarrow \lambda - \lambda_0$ and $u \leftrightarrow u - u_0$.

The following developments involve the energy $(u, \lambda) \mapsto \E(u, \lambda)$ and its differentials at the critical point $(u_0, \lambda_0)$, as well as the fundamental path $\lambda \mapsto u^\ast(\lambda)$ and its derivatives at $\lambda = \lambda_0$. It will therefore be convenient to express $\E$ and $u^\star$ as Taylor expansions with respect to $u$ and $\lambda$.

We start with the Taylor expansion of the energy $\E$. We define its differentials at the critical point. These differentials are stored in a dictionary. Values are indexed with the order of the differentials with respect to $u$ and $\lambda$.

In the following code blocks, `n` is the total order of the differential, `p` is the order of the differential with respect to $u$ and `q = n - p` is the order of the differential with respect to $\lambda$.

In [1]:
from sympy import *

The above statement is extremely poor practice. We need it as `sympy.srepr` automatically drops the namespace. For example

In [2]:
import sympy.abc
srepr(sympy.abc.a + sympy.abc.b)

"Add(Symbol('a'), Symbol('b'))"

Therefore, in order to `eval` strings produced by `sympy.srepr`, as will be done in subsequent chapters, we will need all SymPy object to live in the global namespace.

In [3]:
import lsk

λ, u, v, u_hat, v_hat, w_hat = sympy.symbols(r"\lambda u v \hat{u} \hat{v} \hat{w}")

In [4]:
dE = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        if q == 0:
            dE[p, q] = Symbol(r"\E_{}".format(p))
        else:
            dE[p, q] = Symbol(r"\E_{" + p * "u" + q * "\lambda" + "}")
            
# Remember that the energy is stationary at the critical point (equilibrium)
dE[1, 0] = Number(0)

The dictionary `dE` should be understood as follows: `dE[(2, 1)] == dE[2, 1]` is the symbol $\E_{uu\lambda}$, which stands for $\E_{,uu\lambda}(u_0, \lambda_0; \bullet, \bullet)$. The following symbols have been defined.

In [5]:
#| code-fold: true

_dict = dict()
for n in range(1, 5):
    for p in range(n + 1):
        q = n - p
        _dict[dE[p, q]] = ("\\E_{," + p * "u" 
                           + q * "\\lambda"
                           + "}(u_0, \lambda_0)")
        
lsk.display_latex_dict(_dict, num_cols=3)

<IPython.core.display.Math object>

where

- $\E_\lambda$, $\E_{\lambda\lambda}$, $\E_{\lambda\lambda\lambda}$ and $\E_{\lambda\lambda\lambda\lambda}$ are *scalar quantities*,
- $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$ are *linear forms*,
- $\E_2$, $\E_{uu\lambda}$ and $\E_{uu\lambda\lambda}$ are *bilinear forms*,
- $\E_3$ and $\E_{uuu\lambda}$ are *trilinear forms*,
- $\E_4$  is a *quadrilinear form*.

Note that all these differentials are defined as SymPy *scalars*. A definition as a SymPy *function* (e.g. `E2 = Function(r"\E_2")`) would be more appropriate. However, SymPy would fail to account for multilinearity or symmetry of these forms. Therefore, we use the following trick: all multi-linear forms are defined as scalars, and the standard multiplication operator `*` means function application. In other words, `E2 * (α * u + β * v) * w` (resp. `E2 * u * v - E2 * v * u`) should be understood as `E2(α * u + β * v, w)` (resp. `E2(u, v) - E2(v, u)`). In both cases, the expressions would are correctly simplified.

Whether the symbols in an expression are true scalars or vectors (elements of $U$) should be clear from the context. For example, in the expression: `λ * E2 * u * v`, the first `*` is a true multiplication, while the other `*` refer to function application.

The energy $\E(u, \lambda)$ is now expressed as a Taylor expansion about the critical point.

In [6]:
_f = lambda p, q: binomial(p + q, p) / factorial(p + q)
E = sum(_f(p, q) * dE_pq * u**p * λ**q for (p, q), dE_pq in dE.items())

In [7]:
#| code-fold: true
lsk.display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

We now turn to the definition of the fundamental path $\lambda \mapsto u^\star(\lambda)$. We first define in a dictionary the derivatives of $u^\star$ with respect to $\lambda$, at $\lambda = \lambda_0 = 0$. For example, `du_0[2]` is the symbol $\ddot{u}_0$, which stands for $\D^2 u^\star / \D \lambda^2 \rvert_{\lambda=\lambda_0}$.

In [8]:
du_0 = {n: Symbol("\\" + "".join(n * ("d",)) + "ot{u}_0") for n in range(1, 5)}

_dict = {f"\\frac{{\\D^{k}u^\\ast}}{{\\D \\lambda^{k}}}" : v for k, v in du_0.items()}

In [9]:
#| code-fold: true
lsk.display_latex_dict(_dict, num_cols=4)

<IPython.core.display.Math object>

The fundamental path is then defined through its Taylor expansion.

In [10]:
u_star = sum(λ**n / factorial(n) * v for n, v in du_0.items())

In [11]:
#| code-fold: true
lsk.display_latex_equation(r"u^\star(\lambda)", u_star)

<IPython.core.display.Math object>

## Elimination of the mixed derivatives of the energy {#sec-20230208081549}

Since the fundamental path $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, the various differentials of the energy at the critical point are not linearly independent. To express the relationships between these forms, we define $\mathcal R^\ast(\lambda; \bullet)$ as the jacobian of the energy along the fundamental path $u^\ast(\lambda)$
$$
\mathcal R^\ast(\lambda; \bullet) = \E_{,u}[u^\ast(\lambda), \lambda; \bullet].
$$

In [12]:
R_star = (E.diff(u) * u_hat).subs(u, u_star).expand()

In [13]:
#| code-fold: true
lsk.display_latex_equation(r"\mathcal{R}^\ast(\lambda;" + sympy.latex(u_hat) + ")",
                           R_star.series(λ, 0, 5))

<IPython.core.display.Math object>

Of course, since $\lambda \mapsto u^\ast(\lambda)$ is an equilibrium path, we have $\mathcal R^\ast(\lambda; \bullet) = 0$ for all $\lambda$. Therefore, all coefficients of the above polynomial in $\lambda$ are null, which delivers expressions of $\E_{u\lambda}$, $\E_{u\lambda\lambda}$ and $\E_{u\lambda\lambda\lambda}$

In [14]:
mixed1 = dict()

for q in range(1, 4):
    x = dE[1, q]
    eq = R_star.coeff(λ, q)
    sol = solve(eq, x)[0]
    mixed1[x] = sol

In [15]:
#| code-fold: true
lsk.display_latex_dict(mixed1, num_cols=1)

<IPython.core.display.Math object>

We want to get rid of the remaining mixed derivatives, namely: $\E_{uu\lambda}$, $\E_{uuu\lambda}$ and $\E_{uu\lambda\lambda}$. To do so, we introduce the derivatives $\dot{\E}_2$, $\ddot{\E}_2$ and $\dot{\E}_3$ defined in @sec-20230402152824.

In [16]:
E2_dot, E2_ddot, E3_dot = sympy.symbols("\dot{\E}_2 \ddot{\E}_2 \dot{\E}_3")

E_uu_star = E.diff(u, 2).subs(u, u_star).expand()
E_uuu_star = E.diff(u, 3).subs(u, u_star).expand()

eqs = [Eq(E2_dot, E_uu_star.coeff(λ, 1)),
       Eq(E3_dot, E_uuu_star.coeff(λ, 1)),
       Eq(E2_ddot, 2 * E_uu_star.coeff(λ, 2))]

unknowns = [dE[2, 1], dE[3, 1], dE[2, 2]] # E_uuλ, E_uuuλ, E_uuλλ

mixed2 = dict()
for eq, x in zip(eqs, unknowns):
    mixed2[x] = solve(eq, x)[0].subs(mixed2).expand()

We get the following expressions

In [17]:
#| code-fold: true
lsk.display_latex_dict(mixed2, num_cols=1)

<IPython.core.display.Math object>

Combining the above results allows to fully eliminate the mixed derivatives

In [18]:
#| code-fold: true

mixed = {k: v.subs(mixed2).expand() for k, v in mixed1.items()}
mixed.update(mixed2)

lsk.display_latex_dict(mixed, num_cols=1)

<IPython.core.display.Math object>

These expressions can be plugged into the expansion of the energy.

In [19]:
E = E.subs(mixed).expand()

In [20]:
#| code-fold: true
lsk.display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

From which we deduce the expression of the Hessian of the energy.

In [21]:
E_uu = E.diff(u, 2)

In [22]:
#| code-fold: true
lsk.display_latex_equation(r"\E_{,uu}(u, \lambda)", E_uu)

<IPython.core.display.Math object>

Some of the previous symbolic results are now saved for further use in the subsequent notebooks.

In [23]:
mixed

{\E_{u\lambda}: -\E_2*\dot{u}_0,
 \E_{u\lambda\lambda}: -\E_2*\ddot{u}_0 + \E_3*\dot{u}_0**2 - 2*\dot{\E}_2*\dot{u}_0,
 \E_{u\lambda\lambda\lambda}: -\E_2*\dddot{u}_0 + 3*\E_3*\ddot{u}_0*\dot{u}_0 - \E_4*\dot{u}_0**3 - 3*\ddot{\E}_2*\dot{u}_0 - 3*\ddot{u}_0*\dot{\E}_2 + 3*\dot{\E}_3*\dot{u}_0**2,
 \E_{uu\lambda}: -\E_3*\dot{u}_0 + \dot{\E}_2,
 \E_{uuu\lambda}: -\E_4*\dot{u}_0 + \dot{\E}_3,
 \E_{uu\lambda\lambda}: -\E_3*\ddot{u}_0 + \E_4*\dot{u}_0**2 + \ddot{\E}_2 - 2*\dot{\E}_3*\dot{u}_0}

In [24]:
import json

data = {
    "E" : srepr(E),
    "E_uu" : srepr(E_uu),
    "u_star": srepr(u_star),
    "E_uλ": srepr(mixed[dE[1, 1]]),
    "E_uλλ": srepr(mixed[dE[1, 2]]),
    "E_uλλλ": srepr(mixed[dE[1, 3]]),
    "E_uuλ": srepr(mixed[dE[2, 1]]),
    "E_uuλλ": srepr(mixed[dE[2, 2]]),
    "E_uuuλ": srepr(mixed[dE[3, 1]])
}

with open("setting-up_the_computational_stage.out.json", "w") as f:
    json.dump(data, f)